# Task – ML Facial recognition to detect mood and suggest songs accordingly

# LetsGrowMore Data Science Intern January 2022

# Author - Archana Rathore

# Dataset - FER2013

**Import all the required libraries**

In [17]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#FER2013 dataset
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [18]:
import matplotlib.pyplot as plt
import seaborn as sns

**Importing Deep learning libraries**

In [20]:
from keras.preprocessing.image import load_img , img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense , Input , Dropout , MaxPooling2D , GlobalAveragePooling2D , BatchNormalization ,Flatten , Conv2D , Activation 
from keras.models import Model, Sequential
from tensorflow.keras.optimizers import SGD

In [21]:
train_path = '../input/fer2013/train'
val_path = '../input/fer2013/test'

In [22]:
folder_path = "../input/fer2013"
picture_size = 48

In [23]:
def plot_images(image_dir):
    plt.figure(figsize=(12,12))
    for i in range(1, 10, 1):
        plt.subplot(3,3,i)
        img = load_img(image_dir+"/"+
                  os.listdir(image_dir)[i], target_size=(picture_size, picture_size))
        plt.imshow(img)
    plt.show()    

In [24]:
Expression = "happy"
plot_images(train_path+"/"+Expression)

In [25]:
Expression = "angry"
plot_images(train_path+"/"+Expression)

**Making Training and Validation Data**

In [27]:
datagen_train  = ImageDataGenerator()
datagen_val = ImageDataGenerator()
train_set = datagen_train.flow_from_directory(train_path, target_size=(picture_size,picture_size),
                                             color_mode = "grayscale",
                                             batch_size = 128,
                                             class_mode = "categorical",
                                             shuffle = True)
test_set = datagen_val.flow_from_directory(val_path, target_size=(picture_size,picture_size),
                                             color_mode = "grayscale",
                                             batch_size = 128,
                                             class_mode = "categorical",
                                             shuffle = False)
                                             

**Model Building**

In [29]:
model = Sequential()
#1st CNN layer
model.add(Conv2D(64,(3,3),padding = "same", input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#2nd CNN layer
model.add(Conv2D(128,(3,3),padding = "same"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#3rd CNN layer
model.add(Conv2D(512,(3,3),padding = "same"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#flattening operation
model.add(Flatten())

#Fully connected 1st layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

#Fully connected 2nd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(7, activation='softmax'))

LOSS_FUNCTION = "categorical_crossentropy"
OPTIMIZER = "SGD"
METRICS = ['accuracy']
model.compile(optimizer=OPTIMIZER,loss=LOSS_FUNCTION, metrics=METRICS)
model.summary()

**Fitting the Model with Training and Validation Data**

In [31]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

**Model Checkpoint**

In [32]:
CKPT_PATH = "model_ckpt.h5"
checkpointing_cb = ModelCheckpoint(CKPT_PATH, monitor='val_acc', verbose=1,save_best_only = True, mode='max')

**Early Stopping callbacks**

In [34]:
early_stopping = EarlyStopping(patience=3,monitor='val_loss',verbose=1,min_delta=0,restore_best_weights=True)

**Reduce learning rate**

In [35]:
reduce_learningrate = ReduceLROnPlateau(patience=3,monitor='val_loss',verbose=1,factor=0.2,min_delta=0.0001)

In [36]:
callbacks_list = [checkpointing_cb,early_stopping,reduce_learningrate]

In [37]:
model.compile(loss= LOSS_FUNCTION,optimizer = OPTIMIZER,metrics=METRICS)

In [38]:
EPOCHS = 30
batch_size = 128

history = model.fit_generator(generator=train_set,
                                steps_per_epoch=train_set.n//train_set.batch_size,
                                epochs=EPOCHS,
                                validation_data = test_set,
                                validation_steps = test_set.n//test_set.batch_size,
                                callbacks=callbacks_list
                                )

**Plotting Accuracy & Loss**

In [39]:
plt.style.use('dark_background')

plt.figure(figsize=(20,10))
plt.subplot(1, 2, 1)
plt.suptitle('Optimizer : SGD', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.show()

In [40]:
train_loss , train_acc  =model.evaluate(train_set)
test_loss , test_acc = model.evaluate(test_set)

In [53]:
test_img = load_img('../input/fer2013/test/fear/PrivateTest_10306709.jpg',target_size=(48,48),color_mode = "grayscale")
plt.imshow(test_img)

In [54]:
emotions_dict = {0 : 'Angry', 1 :'Disgusted',2 :'Fearful',3:'Happy',4 :'Neutral',5:'Sad',6:'Surprised'}

In [55]:
test_img = np.expand_dims(test_img,axis=0)
test_img = test_img.reshape(1,48,48,1)
final_result = model.predict(test_img)
final_result = list(final_result[0])
image_index = final_result.index(max(final_result))
print(emotions_dict[image_index])

In [45]:
# model.save('trained_model.h5')

**Test image**

In [137]:
test_img = load_img('../input/fer2013/test/happy/PrivateTest_10613684.jpg',target_size=(48,48),color_mode = "grayscale")
plt.imshow(test_img)

**#Predicting face expression**

In [138]:
test_img = np.expand_dims(test_img,axis=0)
test_img = test_img.reshape(1,48,48,1)
final_result = model.predict(test_img)
final_result = list(final_result[0])
image_index = final_result.index(max(final_result))
print(emotions_dict[image_index])

**Playing Audio**

In [144]:
emotions = emotions_dict[image_index]
import random
n = random.randint(0,2167)
from IPython.display import Audio

#Audio emotion dataset
files = os.listdir('../input/audio-emotions/Emotions/' + emotions)
Audio('../input/audio-emotions/Emotions/' + emotions + '/' + files[n])